<a href="https://colab.research.google.com/github/FrancoArtico/question-generator-LLM/blob/main/FineTunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⬇️ Instalaciones y carga del modelo

In [1]:
!pip install transformers
!pip install datasets

In [2]:
from datasets import load_dataset, Dataset
from google.colab import userdata
from transformers import AutoTokenizer

access_token = userdata.get('HUGGINGFACE_ACCESS_TOKEN')

checkpoint = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          token=access_token,
                                          return_tensors="pt"
)

## 📁 Cargando conjunto de datos

In [3]:
from google.colab import userdata, drive
drive.mount('/content/drive')
dataset = load_dataset("/content/drive/MyDrive/SQAC", split="train") # Dataset's hub has an issue

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 🔍 Inspeccionando el conjunto de datos

In [4]:
print(dataset)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 15036
})


In [5]:
dataset.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [6]:
contexts = dataset[:5]
questions = dataset['question'][:5]

In [7]:
contexts

{'id': ['6cf3dcd6-b5a3-4516-8f9e-c5c1c6b66628',
  '2663226e-e652-43a2-a6ba-c1fd02a1df31',
  '02a1fd0a-b730-4b95-8b07-664d5ac9bd91',
  'a60598d9-70e0-4ccc-a901-a5d21b6aca26',
  'da6bfa6b-d310-4613-b262-465eb95e1724'],
 'title': ['Historia de Japón',
  'Historia de Japón',
  'Historia de Japón',
  'CESS-CAST-A_18964_20000923_rec.txt',
  'CESS-CAST-A_18964_20000923_rec.txt'],
 'context': ['La historia de Japón (日本の歴史 o 日本史, Nihon no rekishi / Nihonshi?) es la sucesión de hechos acontecidos dentro del archipiélago japonés. Algunos de estos hechos aparecen aislados e influenciados por la naturaleza geográfica de Japón como nación insular, en tanto que otra serie de hechos, obedece a influencias foráneas como en el caso del Imperio chino, el cual definió su idioma, su escritura y, también, su cultura política. Asimismo, otra de las influencias foráneas fue la de origen occidental, lo que convirtió al país en una nación industrial, ejerciendo con ello una esfera de influencia y una expansión 

In [8]:
questions

['¿Qué influencia convirtió Japón en una nación industrial?',
 '¿Cuándo se detuvo el expansionismo de Japón?',
 '¿Quién definió el idioma, la escritura y la cultura política del Japón?',
 '¿Qué premio ganó Margarita Fullana?',
 '¿Quién obtuvo la medalla de plata?']

## ⏳ Preprocesamiento de datos

In [9]:
def format_conversation(row):
    user_message = {"role": "user", "content": "Genera exactamente %s preguntas de lectura comprensiva, sin incluir las respuestas ni opciones, sobre el siguiente texto: %s" % (row['num_questions'], row['context'])}
    assistant_message = {"role": "assistant", "content": "Aquí tienes algunas preguntas sobre el texto proporcionado \n %s" % row['questions']}
    row["conversations"] = [user_message, assistant_message]
    return row

def apply_template(row):
    messages = row["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

In [10]:
# Remove unnecessary columns
dataset = dataset.remove_columns(['id','title', 'answers'])

In [11]:
df = dataset.to_pandas()

# Grouping contexts that are the same and enumerate questions for make the prompt
grouped_df = (
    df.groupby('context')
    .agg(
        questions = ('question', lambda qs: "\n".join([f"{i+1}. {q}" for i, q in enumerate(qs)])),
        num_questions = ('question', 'count')
    )
    .reset_index()
)

grouped_df

,context,questions,num_questions
0,""" ¿Cómo ha podido salir así en una familia tan...",1. ¿Cómo era la familia de Chevènement?\n2. ¿A...,2
1,"""Comprendo muy bien la preocupación por los em...",1. ¿Cuándo finalizarán las negociaciones con l...,5
2,"""Convendrá que el Banco Central Europeo (BCE) ...",1. ¿Qué es el BCE?\n2. ¿Cuál es el pronóstico ...,4
3,"""Creo que por lo menos, los restaurantes deber...",1. ¿Cuántos establecimientos estaban engañando...,5
4,"""Cúper es exigente con la plantilla, tiene las...",1. ¿Quién no comparte las mismas ideas que Gas...,3
...,...,...,...
4975,Éstos serán los grandes duelos que se dirimirá...,1. ¿Quiénes son los atletas estadounidenses?\n...,5
4976,"Últimamente, las nodrizas jurásicas la tenían ...",1. ¿Qué tenía entre sus brazos el suicida?\n2....,2
4977,Últimos años \nA pesar de que Morgan fue recib...,1. ¿Dónde fue encarcelado Modyford?\n2. ¿Por q...,3
4978,Últimos años y fallecimiento \nMaciá fue elegi...,1. ¿Cuándo dejó Maciá su mandato provincial?\n...,3


In [12]:
# Convert df to dataset
cq_dataset = Dataset.from_pandas(grouped_df)

# Convert dataset of cq in dataset of conversations
conversation_dataset = cq_dataset.map(format_conversation)
text_dataset = conversation_dataset.remove_columns(['context', 'questions'])
templated_dataset = text_dataset.map(apply_template, batched=True)
templated_dataset['text'][:5]

Map:   0%|          | 0/4980 [00:00<?, ? examples/s]

Map:   0%|          | 0/4980 [00:00<?, ? examples/s]

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenera exactamente 2 preguntas de lectura comprensiva, sin incluir las respuestas ni opciones, sobre el siguiente texto: " ¿Cómo ha podido salir así en una familia tan pacífica?". El ministro del Interior francés, Jean-Pierre Chevènement, ha provocado el escándalo al acusar a la sacrosanta institución. Se ha equivocado al proponer castigar a los padres con la pérdida de las ayudas familiares según los destrozos ciudadanos de sus hijos. Es golpear sólo a un grupo y el más débil. Pero la democracia tiene que buscar una causa a la violencia gratuita juvenil, anárquica o xenófoba.¡Atención al veneno mental de la hora de la sopa!.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAquí tienes algunas preguntas sobre el texto proporcionado \n 1. ¿Cómo era la familia de Chevènement?\n2. ¿A quién quiere c